In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!python -V

Python 3.12.3


In [11]:
import pickle
import numpy as np
import pandas as pd

In [4]:
with open("model.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

/Users/apple/Documents/mlops-dtc-2024/venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/apple/Documents/mlops-dtc-2024/venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ["PULocationID", "DOLocationID"]


def read_data(filename):
    df = pd.read_parquet(filename)

    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype("int").astype("str")

    return df

In [6]:
df = read_data(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet"
)

In [7]:
dicts = df[categorical].to_dict(orient="records")
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [12]:
np.std(y_pred)

6.247488852238703

In [13]:
year = 2023
month = 3
output_file = f"predictions_{year:04d}-{month:02d}.parquet"

In [14]:
df["ride_id"] = f"{year:04d}/{month:02d}_" + df.index.astype("str")

In [15]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['prediction'] = y_pred

In [16]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [26]:
%%bash
ls -lah

total 134128
drwxr-xr-x@  5 apple  staff   160B Jun 19 00:25 .
drwxr-xr-x  13 apple  staff   416B Jun 18 23:55 ..
-rw-r--r--@  1 apple  staff    17K Jun 19 00:15 model.bin
-rw-r--r--   1 apple  staff    65M Jun 19 00:25 predictions_2023-03.parquet
-rw-r--r--@  1 apple  staff   5.8K Jun 19 00:27 starter.ipynb
